In [1]:
from fertilizer_recommender.infrastructure.utils.root_finder import get_repository_root

root = get_repository_root()

print("Root of this project:", root)


Root of this project: /Users/surelmanda/Downloads/ml-projects/Clean-Architecture-MLops/fertilizer_recommender


In [2]:
from fertilizer_recommender.infrastructure.utils.root_finder import get_repository_root

log_file = get_repository_root() / "logs" / "fertilizer_recommender.log"
print("Log file exists:", log_file.exists())
print("Log file path:", log_file)


Log file exists: False
Log file path: /Users/surelmanda/Downloads/ml-projects/Clean-Architecture-MLops/fertilizer_recommender/logs/fertilizer_recommender.log


In [ ]:
from fertilizer_recommender.infrastructure.observability.logger import setup_logger
from loguru import logger

setup_logger(
    project_name="fertilizer_recommender",
    environment="notebook",
    level="DEBUG",
)

logger.info("Logger initialisé depuis le notebook")





2025-12-15 11:53:19 | INFO | fertilizer_recommender | notebook | 1481428342:<module>:10 | Logger initialisé depuis le notebook


In [7]:
from fertilizer_recommender.domain.services.metric_service import map_at_k
from fertilizer_recommender.infrastructure.utils.config_loader import load_yaml_config
from fertilizer_recommender.infrastructure.utils.root_finder import get_repository_root



root = get_repository_root()

config_path = root / "configs/training.yaml"

cfg = load_yaml_config(config_path)

logger.info(f"Config project.name = {cfg['project']['name']}")
logger.info(f"Target col = {cfg['data']['target_col']} | top_k = {cfg['training']['top_k']}")

# Test métrique MAP@3
y_true = ["A", "B", "C"]
y_pred = [
    ["A", "B", "C"],  # 1.0
    ["C", "B", "A"],  # B en rank2 => 0.5
    ["D", "E", "F"],  # 0.0
]
score = map_at_k(y_true, y_pred, k=3)
logger.info(f"MAP@3 test = {score:.4f} (attendu ~0.5000)")


2025-12-15 12:28:53 | INFO | fertilizer_recommender | notebook | 1113757861:<module>:13 | Config project.name = fertilizer_recommender
2025-12-15 12:28:53 | INFO | fertilizer_recommender | notebook | 1113757861:<module>:14 | Target col = Fertilizer Name | top_k = 3
2025-12-15 12:28:53 | INFO | fertilizer_recommender | notebook | 1113757861:<module>:24 | MAP@3 test = 0.5000 (attendu ~0.5000)


In [4]:
from fertilizer_recommender.domain.entities.fertilizer_label import FertilizerLabel

label = FertilizerLabel("Urea")
print(label)

Urea


In [5]:
from fertilizer_recommender.domain.entities.recommendation import Recommendation

rec = Recommendation([
    FertilizerLabel("Urea"),
    FertilizerLabel("DAP"),
    FertilizerLabel("14-35-14"),
])

rec.as_strings()
rec.as_kaggle_string()

'Urea DAP 14-35-14'

In [6]:
from fertilizer_recommender.infrastructure.data_sources.schema_inference import infer_feature_types
""""
num_feats, cat_feats = infer_feature_types(
    train_df,
    target_col=cfg["data"]["target_col"],
    id_col=cfg["data"]["id_col"],
)

num_feats, cat_feats
"""

'"\nnum_feats, cat_feats = infer_feature_types(\n    train_df,\n    target_col=cfg["data"]["target_col"],\n    id_col=cfg["data"]["id_col"],\n)\n\nnum_feats, cat_feats\n'

In [ ]:
"""
enc = TargetEncoder().fit(["A", "B", "A", "C"])
enc.transform(["C", "A"])
enc.inverse_transform([0, 2])
"""

In [ ]:
"""
from fertilizer_recommender.infrastructure.observability.tracing import trace
import time

with trace("sleep_test"):
    time.sleep(1)
"""

In [ ]:
from fertilizer_recommender.infrastructure.utils.seed import set_global_seed
set_global_seed(42)

In [ ]:
from fertilizer_recommender.composition_root import build_prepare_dataset_use_case

use_case = build_prepare_dataset_use_case("../configs/training.yaml")
train_df, test_df = use_case.execute()
train_df.head()

In [ ]:
from fertilizer_recommender.composition_root import (
    build_prepare_dataset_use_case,
    build_train_with_cv_use_case,
)

prepare_uc = build_prepare_dataset_use_case(load_yaml_config("configs/training.yaml"))
train_df, test_df = prepare_uc.execute()

train_uc = build_train_with_cv_use_case(
    model_name="catboost",
    X=train_df,
)

result = train_uc.execute(
    X_df=train_df.drop(columns=["Fertilizer Name"]),
    y=train_df["Fertilizer Name"],
    run_name="catboost_cv",
    params={"model": "catboost"},
)